<a href="https://colab.research.google.com/github/MariaSparrow/PySpark/blob/main/pyspark_vm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Установка библиотеки PySpark

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=ca7292502499a586226660154eb33f9c8621b07ce7319f6dccbe8e2ec22ee330
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = spark.read.csv('/content/drive/MyDrive/notebooks/input/дз_iris.csv',
                    header=True, 
                    sep=',', 
                    inferSchema=True,
                    quote=""
                    )

In [ ]:
df.show()

+-------------+---------------+----------------+---------------+------------+
|"sepal.length|""sepal.width""|""petal.length""|""petal.width""|""variety"""|
+-------------+---------------+----------------+---------------+------------+
|         "5.1|            3.5|             1.4|            0.2| ""Setosa"""|
|         "4.9|            3.0|             1.4|            0.2| ""Setosa"""|
|         "4.7|            3.2|             1.3|            0.2| ""Setosa"""|
|         "4.6|            3.1|             1.5|            0.2| ""Setosa"""|
|           "5|            3.6|             1.4|            0.2| ""Setosa"""|
|         "5.4|            3.9|             1.7|            0.4| ""Setosa"""|
|         "4.6|            3.4|             1.4|            0.3| ""Setosa"""|
|           "5|            3.4|             1.5|            0.2| ""Setosa"""|
|         "4.4|            2.9|             1.4|            0.2| ""Setosa"""|
|         "4.9|            3.1|             1.5|            0.1|

In [ ]:
import pyspark.sql.functions as F

df = df.withColumnRenamed('"sepal.length', 'sepal_length') \
       .withColumnRenamed('""sepal.width""', 'sepal_width') \
       .withColumnRenamed('""petal.length""', 'petal_length') \
       .withColumnRenamed('""petal.width""', 'petal_width') \
       .withColumnRenamed('""variety"""', 'variety')
df.show(5)

+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|    variety|
+------------+-----------+------------+-----------+-----------+
|        "5.1|        3.5|         1.4|        0.2|""Setosa"""|
|        "4.9|        3.0|         1.4|        0.2|""Setosa"""|
|        "4.7|        3.2|         1.3|        0.2|""Setosa"""|
|        "4.6|        3.1|         1.5|        0.2|""Setosa"""|
|          "5|        3.6|         1.4|        0.2|""Setosa"""|
+------------+-----------+------------+-----------+-----------+
only showing top 5 rows



In [ ]:
df=df.withColumn('variety', F.regexp_replace('variety', '\"',''))\
.withColumn('sepal_length', F.regexp_replace('sepal_length', '\"',''))

df.show(10)

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|           5|        3.6|         1.4|        0.2| Setosa|
|         5.4|        3.9|         1.7|        0.4| Setosa|
|         4.6|        3.4|         1.4|        0.3| Setosa|
|           5|        3.4|         1.5|        0.2| Setosa|
|         4.4|        2.9|         1.4|        0.2| Setosa|
|         4.9|        3.1|         1.5|        0.1| Setosa|
+------------+-----------+------------+-----------+-------+
only showing top 10 rows



In [ ]:
df.groupBy('variety') \
  .agg(F.min("sepal_length").alias("min_sl"), 
       F.max("sepal_length").alias("max_sl"),
       F.avg("sepal_length").alias("avg_sl"),
       F.min("sepal_width").alias("min_sw"), 
       F.max("sepal_width").alias("max_sw"),
       F.avg("sepal_width").alias("avg_sw"),
       F.min("petal_length").alias("min_pl"), 
       F.max("petal_length").alias("max_pl"),
       F.avg("petal_length").alias("avg_pl"),
       F.min("petal_width").alias("min_pw"), 
       F.max("petal_width").alias("max_pw"),
       F.avg("petal_width").alias("avg_pw")
       ).show()

+----------+------+------+-----------------+------+------+------------------+------+------+------------------+------+------+------------------+
|   variety|min_sl|max_sl|           avg_sl|min_sw|max_sw|            avg_sw|min_pl|max_pl|            avg_pl|min_pw|max_pw|            avg_pw|
+----------+------+------+-----------------+------+------+------------------+------+------+------------------+------+------+------------------+
|    Setosa|   4.3|   5.8|5.005999999999999|   2.3|   4.4| 3.428000000000001|   1.0|   1.9|1.4620000000000002|   0.1|   0.6|0.2459999999999999|
|Versicolor|   4.9|     7|            5.936|   2.0|   3.4|2.7700000000000005|   3.0|   5.1|              4.26|   1.0|   1.8|1.3259999999999998|
| Virginica|   4.9|   7.9|6.587999999999998|   2.2|   3.8|2.9739999999999998|   4.5|   6.9|             5.552|   1.4|   2.5|             2.026|
+----------+------+------+-----------------+------+------+------------------+------+------+------------------+------+------+------------

Другой способ привести файл в порядок

In [ ]:
with open('/content/drive/MyDrive/notebooks/input/дз_iris.csv') as file1:
  with open('/content/drive/MyDrive/notebooks/input/iris_2.csv', 'w') as file2:
    for line in file1:
      file2.write(line.replace('"', ''))

df_2 = spark.read.csv('/content/drive/MyDrive/notebooks/input/iris_2.csv', inferSchema=True, header=True)

df_2 = df_2.withColumnRenamed('sepal.length', 'sepal_length')
df_2 = df_2.withColumnRenamed('sepal.width', 'sepal_width')
df_2 = df_2.withColumnRenamed('petal.length', 'petal_length')
df_2 = df_2.withColumnRenamed('petal.width', 'petal_width')

In [ ]:
df_2.groupBy('variety') \
  .agg(F.min("sepal_length").alias("min_sl"), 
       F.max("sepal_length").alias("max_sl"),
       F.avg("sepal_length").alias("avg_sl"),
       F.min("sepal_width").alias("min_sw"), 
       F.max("sepal_width").alias("max_sw"),
       F.avg("sepal_width").alias("avg_sw"),
       F.min("petal_length").alias("min_pl"), 
       F.max("petal_length").alias("max_pl"),
       F.avg("petal_length").alias("avg_pl"),
       F.min("petal_width").alias("min_pw"), 
       F.max("petal_width").alias("max_pw"),
       F.avg("petal_width").alias("avg_pw")
       ).show()

+----------+------+------+-----------------+------+------+------------------+------+------+------------------+------+------+------------------+
|   variety|min_sl|max_sl|           avg_sl|min_sw|max_sw|            avg_sw|min_pl|max_pl|            avg_pl|min_pw|max_pw|            avg_pw|
+----------+------+------+-----------------+------+------+------------------+------+------+------------------+------+------+------------------+
| Virginica|   4.9|   7.9|6.587999999999998|   2.2|   3.8|2.9739999999999998|   4.5|   6.9|             5.552|   1.4|   2.5|             2.026|
|    Setosa|   4.3|   5.8|5.005999999999999|   2.3|   4.4| 3.428000000000001|   1.0|   1.9|1.4620000000000002|   0.1|   0.6|0.2459999999999999|
|Versicolor|   4.9|   7.0|            5.936|   2.0|   3.4|2.7700000000000005|   3.0|   5.1|              4.26|   1.0|   1.8|1.3259999999999998|
+----------+------+------+-----------------+------+------+------------------+------+------+------------------+------+------+------------